In [ ]:
!pip install openai
!pip install datasets
import pandas as pd
import random
import openai

# Configura tu API key de OpenAI
openai.api_key = 'YourApi'

def cargar_recetas():
    """Carga y prepara el DataFrame de recetas desde un archivo CSV."""
    try:
        df = pd.read_csv("hf://datasets/somosnlp/RecetasDeLaAbuela/recetasdelaabuela.csv")
        df = df.rename(columns={
            'Nombre': 'Título',
            'Ingredientes': 'Ingredientes',
            'Pasos': 'Pasos'
        })
        df['Categoría'] = 'Comida'
        columns_to_drop = [
            'URL', 'Pais', 'Duracion', 'Porciones', 'Calorias',
            'Categoria', 'Contexto', 'Valoracion y Votos',
            'Comensales', 'Tiempo', 'Dificultad',
            'Categoria 2', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18'
        ]
        df = df.drop(columns=columns_to_drop, errors='ignore')
        df = df.fillna('')
        return df
    except Exception as e:
        print(f"Error al cargar las recetas: {str(e)}")
        return None

def traducir_a_espanol(texto):
    """Traduce un texto al español utilizando la API de OpenAI."""
    try:
        response = openai.Completion.create(
            engine="davinci",
            prompt=f"Traduce esto al español: {texto}",
            max_tokens=150
        )
        return response.choices[0].text.strip()
    except Exception as e:
        return f"Error en la traducción: {str(e)}"

def formatear_lista_en_ascendente(lista):
    """Formatea una lista en orden ascendente."""
    return "\n".join(f"{i+1}. {elemento.strip()}" for i, elemento in enumerate(lista))

def buscar_receta_por_nombre(nombre, df):
    """Busca una receta por nombre en el DataFrame."""
    resultados = df[df['Título'].str.contains(nombre, case=False, na=False)]
    if len(resultados) > 0:
        if len(resultados) == 1:
            receta = resultados.iloc[0]
            return mostrar_receta(receta), resultados
        else:
            opciones = "\n".join([f"{i+1}. {receta['Título']}" for i, (_, receta) in enumerate(resultados.iterrows(), start=1)])
            return f"Se encontraron varias recetas con ese nombre. Por favor elige una:\n{opciones}", resultados
    else:
        return "Lo siento, no encontré ninguna receta que coincida con tu búsqueda.", None

def mostrar_receta(receta):
    """Muestra una receta específica."""
    ingredientes = receta['Ingredientes'].split(",")
    try:
        pasos = eval(receta['Pasos']) if isinstance(eval(receta['Pasos']), list) else receta['Pasos'].split(". ")
    except Exception as e:
        pasos = receta['Pasos'].split(". ")
    ingredientes_str = "\n".join(ingredientes)
    pasos_str = formatear_lista_en_ascendente(pasos)
    return f"Receta: {receta['Título']}\n\nIngredientes:\n{ingredientes_str}\n\nPasos:\n{pasos_str}\n\nCategoría: {receta['Categoría']}"

def mostrar_recetas(df, num_recetas=10, excluidas=[]):
    """Muestra un número específico de recetas, excluyendo algunas."""
    recetas_disponibles = df[~df['Título'].isin(excluidas)]
    recetas = recetas_disponibles['Título'].sample(min(num_recetas, len(recetas_disponibles))).tolist()
    for i, receta in enumerate(recetas, start=1):
        print(f"{i}. {receta}")
    return recetas

def guardar_historial_en_csv(historial):
    """Guarda el historial de recetas consultadas en un archivo CSV."""
    try:
        historial_data = []
        for receta in historial:
            ingredientes = receta['Ingredientes'].split(",")
            try:
                pasos = eval(receta['Pasos']) if isinstance(eval(receta['Pasos']), list) else receta['Pasos'].split(". ")
            except Exception as e:
                pasos = receta['Pasos'].split(". ")
            historial_data.append({
                'Título': receta['Título'],
                'Ingredientes': "\n".join(ingredientes),
                'Pasos': "\n".join(pasos)
            })
        df_historial = pd.DataFrame(historial_data)
        df_historial.to_csv('historial_recetas.csv', index=False, encoding='utf-8-sig')
        print("Historial guardado exitosamente en 'historial_recetas.csv'.")
    except Exception as e:
        print(f"Error al guardar el historial: {str(e)}")

def chatbot():
    """Simula la interacción con el usuario."""
    df = cargar_recetas()
    if df is None:
        print("No se pudo cargar el archivo de recetas. Por favor revisa el archivo y vuelve a intentarlo.")
        return

    historial_busquedas = []
    recetas_mostradas = []

    print("¡Hola! Soy tu asistente de recetas de comida.")
    print("Aquí tienes algunas recetas populares de comida:")

    nuevas_recetas = mostrar_recetas(df, 10)
    recetas_mostradas.extend(nuevas_recetas)

    print("Elige el número de la receta que te interese o escribe 'buscar [nombre de la receta]' para más detalles.")


    while True:
        user_input = input("Usuario: ").strip().lower()

        if user_input == 'sí':
            print("Aquí tienes más recetas de comida:")
            nuevas_recetas = mostrar_recetas(df, 15, recetas_mostradas)
            recetas_mostradas.extend(nuevas_recetas)
            print("Elige el número de la receta que te interese o escribe 'buscar [nombre de la receta]' para más detalles.")

        elif user_input == 'no':
            print("¡Hasta luego! Gracias por usar el Chatbot de Recetas de Comida.")
            guardar_historial_en_csv(historial_busquedas)
            break

        elif 'buscar' in user_input:
            receta_nombre = user_input.replace('buscar', '').strip()
            respuesta, resultados = buscar_receta_por_nombre(receta_nombre, df)
            print(respuesta)
            if resultados is not None:
                seleccion = input("Usuario: ").strip()
                if seleccion.isdigit():
                    indice = int(seleccion) - 1
                    if 0 <= indice < len(resultados):
                        receta_elegida = resultados.iloc[indice]
                        print(mostrar_receta(receta_elegida))
                        historial_busquedas.append(receta_elegida)
                    else:
                        print("El número de receta seleccionado no es válido.")

        elif user_input.isdigit():
            indice = int(user_input) - 1
            if 0 <= indice < len(recetas_mostradas):
                receta_elegida = df[df['Título'] == recetas_mostradas[indice]].iloc[0]
                print(mostrar_receta(receta_elegida))
                historial_busquedas.append(receta_elegida)
                print("¿Deseas realizar otra selección? (sí/no)")
                respuesta = input("Usuario: ").strip().lower()
                if respuesta == 'no':
                    print("¡Hasta luego! Gracias por usar el Chatbot de Recetas de Comida.")
                    guardar_historial_en_csv(historial_busquedas)
                    break
            else:
                print("El número de receta seleccionado no es válido.")

        else:
            print("No entendí tu solicitud. ¿Puedes intentar de nuevo?")

if __name__ == "__main__":
    print("Bienvenido al Chatbot de Recetas de Comida.")
    chatbot()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Succ